# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [39]:
!pip install pyorient

2790.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [40]:
import pyorient

In [ ]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [ ]:
print(client.db_list())

## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [ ]:
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [ ]:
name = client.db_open("gods", "root", "root")

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [ ]:
vertex_content = {"name": "Zeus", "symbol": "thunder"}
sql_command = f"CREATE VERTEX V CONTENT {vertex_content}"
result = client.command(sql_command)

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [ ]:
sql_command = "SELECT * FROM V"
vertices = client.command(sql_command)
for vertex in vertices:
    print(vertex)

**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [ ]:
vertices_content = [
    {"name": "Héra", "symbol": "tiara"},
    {"name": "Poséidon", "symbol": "trident"},
    {"name": "Athena", "symbol": "helmet"},
    {"name": "Arès", "symbol": "weapons"}
]

for vertex_content in vertices_content:
    sql_command = f"CREATE VERTEX V CONTENT {vertex_content}"
    result = client.command(sql_command)
    print("Vertex created:", result)

**Q:** Display all vertices with name = `Arès`

In [ ]:
sql_command = "SELECT * FROM V WHERE name = 'Arès'"
vertices = client.command(sql_command)
print("Vertices with name 'Arès':")
for vertex in vertices:
    print(vertex)

**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [ ]:
edge_content = {"kind": "sibling"}
sql_command = (
    "CREATE EDGE Sibling FROM (SELECT FROM V WHERE name = 'Zeus') "
    "TO (SELECT FROM V WHERE name = 'Poséidon') CONTENT "
    f"{edge_content}"
)
result = client.command(sql_command)

print("Edge created:", result)

**Q:** Redisplay all vertices, discuss.

In [ ]:
sql_command = "SELECT * FROM V"
vertices = client.command(sql_command)
for vertex in vertices:
    print(vertex)

**Q:** Display all edges. They are contained in the class `E`

In [ ]:
sql_command = "SELECT * FROM E"
edges = client.command(sql_command)
for edge in edges:
    print(edge)

Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [ ]:
edge_commands = [
    "CREATE EDGE Sibling FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Héra')",
    "CREATE EDGE Father FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Arès')",
    "CREATE EDGE Father FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Athena')",
    "CREATE EDGE Mother FROM (SELECT FROM V WHERE name = 'Héra') TO (SELECT FROM V WHERE name = 'Arès')",
    "CREATE EDGE Sibling FROM (SELECT FROM V WHERE name = 'Héra') TO (SELECT FROM V WHERE name = 'Zeus')",
    "CREATE EDGE Sibling FROM (SELECT FROM V WHERE name = 'Poséidon') TO (SELECT FROM V WHERE name = 'Zeus')"
]

for edge_command in edge_commands:
    client.command(edge_command)

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [ ]:
sql_command = "SELECT EXPAND(out()) FROM V WHERE name = 'Zeus'"
outgoing_vertices = client.command(sql_command)
for vertex in outgoing_vertices:
    print(vertex)

**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [ ]:
sql_command = "SELECT expand(out()) FROM (SELECT expand(in()) FROM E WHERE kind = 'father')"
vertices_with_father = client.command(sql_command)
for vertex in vertices_with_father:
    print(vertex)

**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [ ]:
sql_command = "SELECT expand(in()) FROM (SELECT expand(out()) FROM E WHERE kind = 'mother')"
mothers_vertices = client.command(sql_command)
for vertex in mothers_vertices:
    print(vertex)

**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [ ]:
sql_command = "SELECT expand(out()) FROM (SELECT expand(in()) FROM E WHERE kind = 'sibling' AND out.name = 'Zeus')"
siblings_vertices = client.command(sql_command)
for vertex in siblings_vertices:
    print(vertex)

## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [ ]:
db_name = "eCommerce"

client.db_create(db_name, pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_PLOCAL)
client.db_open(db_name, "root", "root")

client.command("CREATE CLASS Product EXTENDS V")
client.command("CREATE CLASS Clients EXTENDS V")
client.command("CREATE CLASS Purchases EXTENDS E")

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [ ]:
products = ["spaghetti", "bolognese sauce", "cheese", "apple"]
for product in products:
    client.command(f"CREATE VERTEX Product SET name = '{product}'")

**Q:** Create the following people: `peter`, `meredith`.

In [ ]:
peoples =["peter", "meredith"]
for client in peoples:
    client.command(f"CREATE VERTEX Clients SET nickname = '{client}'")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [ ]:
purchases = [
    {"person": "peter", "products": ["spaghetti", "cheese"], "date": "20/01/2016"},
    {"person": "meredith", "products": ["cheese", "apple", "bolognese sauce"], "date": "22/01/2016"},
    {"person": "peter", "products": ["spaghetti", "bolognese sauce"], "date": "27/01/2016"}
]

for purchase in purchases:
    person = purchase["person"]
    products = purchase["products"]
    date = purchase["date"]
    
    # Create a vertex for the person if not exists
    client.command(f"CREATE VERTEX Clients SET nickname = '{person}'")
    
    # Create edges for each product purchased by the person
    for product in products:
        # Create a vertex for the product if not exists
        client.command(f"CREATE VERTEX Product SET name = '{product}'")
        
        # Create an edge for the purchase
        client.command(f"CREATE EDGE Purchases FROM (SELECT FROM Person WHERE nickname = '{person}') TO (SELECT FROM Product WHERE name = '{product}') SET date = '{date}'")

**Q:** Who bought Bolognese sauce?

In [ ]:
sql_command = (
    "SELECT expand(out('Purchases').in) "
    "FROM (SELECT FROM Product WHERE name = 'bolognese sauce')"
)

result = client.command(sql_command)

for person in result:
    print(person.nickname)

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [ ]:
sql_command = (
    "SELECT expand(out('Purchases').in('Purchases')) "
    "FROM (SELECT FROM Product WHERE name = 'bolognese sauce')"
)

result = client.command(sql_command)

for product in result:
    print(product.name)

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.